In [3]:
!pip install uszipcode

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from uszipcode import SearchEngine

# 获取数据
owners_df = pd.read_csv("w.csv")

print(owners_df.head())

<ipython-input-3-d57d0e6b6a8c>:10: DtypeWarning: Columns (5,35) have mixed types. Specify dtype option on import or set low_memory=False.
  owners_df = pd.read_csv("w.csv")


   Unnamed: 0  disasterNumber state            county          city zipCode  \
0           0            1439    TX  Aransas (County)  ARANSAS PASS   78335   
1           1            1439    TX  Aransas (County)  ARANSAS PASS   78336   
2           2            1439    TX  Aransas (County)        FULTON   78358   
3           3            1439    TX  Aransas (County)        FULTON   78381   
4           4            1439    TX  Aransas (County)      ROCKPORT   78331   

   validRegistrations  averageFemaInspectedDamage  totalInspected  \
0                   4                     1345.01               3   
1                  68                     3082.82              63   
2                  20                     4722.34              18   
3                   1                     2578.30               1   
4                   1                        0.00               0   

   totalDamage  ...                updateDate              closeoutDate  \
0      5380.02  ...  2013-04-01T00:

In [4]:
from uszipcode import SearchEngine
import pandas as pd

# 使用uszipcode来获取经纬度
search = SearchEngine()

def get_lat_lon(zip_code):
    result = search.by_zipcode(zip_code)
    if result and result.lat and result.lng:
        return pd.Series((result.lat, result.lng))
    else:
        return pd.Series((None, None))

# 清理 totalDamage 和 totalApprovedIhpAmount 为0的行
owners_df = owners_df[(owners_df["totalDamage"] > 0) & (owners_df["totalApprovedIhpAmount"] > 0)]

# 补充经纬度
owners_df[['latitude', 'longitude']] = owners_df['zipCode'].apply(get_lat_lon)

# 过滤在边界外的数据
within_bounds = (owners_df['latitude'].between(24.396308, 49.384358) &
                 owners_df['longitude'].between(-125.000000, -66.934570))
owners_df = owners_df[within_bounds]

# 数据分组和总结
disaster_freq = owners_df.groupby(['state', 'incidentType']).size().reset_index(name='disaster_frequency')

# 按照州和灾害类型对其他列进行汇总
owners_summary_state = owners_df.groupby(['state', 'incidentType']).agg(
    total_damage_sum=('totalDamage', 'sum'),
    total_approved_amount_sum=('totalApprovedIhpAmount', 'sum'),
    total_max_grants_sum=('totalMaxGrants', 'sum')
).reset_index()

# 将灾害频率合并到总结数据中
owners_summary_state = pd.merge(owners_summary_state, disaster_freq, on=['state', 'incidentType'])

# 输出结果
print(owners_summary_state)

Download /root/.uszipcode/simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!
    state  incidentType  total_damage_sum  total_approved_amount_sum  \
0      AK         Flood      6.099900e+02               3.843990e+03   
1      AL     Hurricane      4.961169e+08               2.974380e+08   
2      AL  Severe Storm      3.089936e+08               1.207315e+08   
3      AL       Tornado      4.068801e+06               1.095325e+06   
4      AR         Flood      1.554136e+07               8.741423e+06   
..    ...           ...               ...                        ...   
125    WI  Severe Storm      1.065143e+08               9.578034

In [5]:
owners_summary_zip = owners_df.groupby(['state', 'zipCode', 'latitude', 'longitude', 'disasterName', 'incidentType', 'totalMaxGrants']).agg(
    total_damage_sum=('totalDamage', 'sum'),
    total_approved_amount_sum=('totalApprovedIhpAmount', 'sum'),
    total_max_grants_sum=('totalMaxGrants', 'sum')
).reset_index()

In [6]:
# 输出到 CSV 文件
owners_summary_state.to_csv("owners_summary.csv", index=False)
owners_summary_zip.to_csv("owners_summary_zip.csv", index=False)